## 1. 학습데이터 구성

In [3]:
# MIDI 포맷 형식 변환
import os
from mido import MidiFile, MidiTrack, Message

def convert_format0_to_format1_preserve_tracks(input_file, output_file):
    midi = MidiFile(input_file)

    if midi.type == 1:
        print(f"{input_file}는 이미 포맷 1입니다.")
        return

    new_midi = MidiFile(type=1, ticks_per_beat=midi.ticks_per_beat)

    channel_tracks = {}

    for message in midi.tracks[0]:
        if not message.is_meta:
            # Use message.channel to group by channel
            channel = message.channel if hasattr(message, 'channel') else 0
            if channel not in channel_tracks:
                channel_tracks[channel] = MidiTrack()  # Create new track for each channel
                new_midi.tracks.append(channel_tracks[channel])
            channel_tracks[channel].append(message)
        else:
            # Append meta messages to all tracks (or create a dedicated track)
            for track in channel_tracks.values():
                track.append(message)

    new_midi.save(output_file)
    print(f"{input_file} -> {output_file}: 포맷 1로 변환 완료!")

def process_folder(folder_path, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for file_name in os.listdir(folder_path):
        input_file = os.path.join(folder_path, file_name)
        
        if os.path.isfile(input_file) and file_name.lower().endswith('.mid'):
            output_file = os.path.join(output_folder, file_name)
            convert_format0_to_format1_preserve_tracks(input_file, output_file)

input_folder = "train_data"  # 입력 폴더 경로
output_folder = "retry_converted_data"  # 출력 폴더 경로

process_folder(input_folder, output_folder)


train_data\BM_CR1_00052.mid -> retry_converted_data\BM_CR1_00052.mid: 포맷 1로 변환 완료!
train_data\BM_CR1_00114.mid -> retry_converted_data\BM_CR1_00114.mid: 포맷 1로 변환 완료!
train_data\BM_CR1_02016.mid -> retry_converted_data\BM_CR1_02016.mid: 포맷 1로 변환 완료!
train_data\BM_CR1_02045.mid -> retry_converted_data\BM_CR1_02045.mid: 포맷 1로 변환 완료!
train_data\BM_CR1_02069.mid -> retry_converted_data\BM_CR1_02069.mid: 포맷 1로 변환 완료!
train_data\BM_CR1_02103.mid -> retry_converted_data\BM_CR1_02103.mid: 포맷 1로 변환 완료!
train_data\BM_CR1_02115.mid -> retry_converted_data\BM_CR1_02115.mid: 포맷 1로 변환 완료!
train_data\BM_CR1_02124.mid -> retry_converted_data\BM_CR1_02124.mid: 포맷 1로 변환 완료!
train_data\BM_CR1_02133.mid -> retry_converted_data\BM_CR1_02133.mid: 포맷 1로 변환 완료!
train_data\BM_CR1_02139.mid -> retry_converted_data\BM_CR1_02139.mid: 포맷 1로 변환 완료!
train_data\BM_CR1_02141.mid -> retry_converted_data\BM_CR1_02141.mid: 포맷 1로 변환 완료!
train_data\BM_CR1_02152.mid -> retry_converted_data\BM_CR1_02152.mid: 포맷 1로 변환 완료!
trai

In [76]:
# midi 파일 서로 확인하기
from mido import MidiFile

file_1 = MidiFile('3.mid')
file_2 = MidiFile('4.mid')

def analyze_midi(file):
    return {
        "tracks": len(file.tracks),
        "type": file.type,
        "length": file.length,
        "ticks_per_beat": file.ticks_per_beat,
}

analysis_1 = analyze_midi(file_1)
analysis_2 = analyze_midi(file_2)

print("File 1:", analysis_1)
print("File 2:", analysis_2)

File 1: {'tracks': 1, 'type': 1, 'length': 38.0, 'ticks_per_beat': 480}
File 2: {'tracks': 1, 'type': 0, 'length': 38.0, 'ticks_per_beat': 480}


## 2. 학습용 이미지 파일 생성

In [4]:
from music21 import *
import pathlib
import matplotlib.pyplot as plt
import csv

env = environment.Environment()  
musescore_path = r"C:\Program Files\MuseScore 4\bin\MuseScore4.exe"  # MuseScore 실행 파일의 경로로 변경하세요.  
env['musicxmlPath'] = musescore_path  
env['musescoreDirectPNGPath'] = musescore_path 

file_name = 'BP_CR2_01853'
# MIDI 파일 불러오기
score = converter.parse(f'handied_data/{file_name}.mid')

key_signature = score.analyze('key') # 조성
key_signature = key.KeySignature(0)
time_signature = score.getElementsByClass(meter.TimeSignature) 

if len(time_signature) > 0:
    time_signature = time_signature[0]
else: 
    time_signature = meter.TimeSignature('4/4') # 박자
    
note_list = []
beat_list = []
notes = []
for element in score.flat:
    if isinstance(element, note.Note):
        notes.append(element)  # 개별 음표 추가
    elif isinstance(element, chord.Chord):
        notes.extend(element.notes)  # 화음을 구성하는 음 추가
    elif isinstance(element, note.Rest):
        notes.append(element)  # 쉼표 추가


for n in notes:
    if isinstance(n, note.Rest):
        note_list.append('Rest')  # 쉼표를 'Rest'로 표시
        beat_list.append(n.duration.quarterLength)
    elif hasattr(n, 'nameWithOctave'):
        note_list.append(n.nameWithOctave)
        beat_list.append(n.duration.quarterLength)


notes = [value for value in zip(note_list, beat_list) ]

part = stream.Part()
part.append(instrument.Piano())
part.append(time_signature)
part.append(key_signature)

metronome_marks = score.metronomeMarkBoundaries()
    
for start, end, metronome_mark in metronome_marks:
    if metronome_mark is not None:
        new_metronome_mark = tempo.MetronomeMark(number=metronome_mark.getQuarterBPM())
        part.append(new_metronome_mark)

for pitch, duration in notes:
    if pitch == 'Rest':  # 쉼표인 경우
        n = note.Rest()
    else:  # 음표인 경우
        n = note.Note(pitch)
    n.duration.quarterLength = duration
    part.append(n)

new_score = stream.Score()

new_score.append(part)

# PNG 파일 저장
output_png_path = f'handied_data/{file_name}.png'
new_score.write('musicxml.png', output_png_path)
print(f"PNG 파일 저장 완료: {output_png_path}")

# CSV 파일 저장
output_csv_path = f'handied_data/{file_name}.csv'
with open(output_csv_path, 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['Pitch/Rest', 'Duration'])  # 헤더 작성
    csvwriter.writerows(notes)  # 음표 데이터 작성
print(f"CSV 파일 저장 완료: {output_csv_path}")

C:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\interactiveshell.py:3526: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  exec(code_obj, self.user_global_ns, self.user_ns)


PNG 파일 저장 완료: handied_data/BP_CR2_01853.png
CSV 파일 저장 완료: handied_data/BP_CR2_01853.csv


In [15]:
import csv

csv_file = 'handied_data/BP_CR2_01853.csv'
notes = []

with open(csv_file, 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        notes.append((row['Pitch/Rest'], row['Duration']))

print(notes)


[('A3', '0.5'), ('A3', '0.5'), ('A3', '0.25'), ('Rest', '0.25'), ('A3', '0.5'), ('A3', '0.5'), ('A3', '0.5'), ('A3', '0.5'), ('A3', '0.5'), ('A3', '0.25'), ('Rest', '1.25'), ('A3', '0.5'), ('A3', '1/3'), ('A3', '1/3'), ('Rest', '1/12'), ('A3', '1/3'), ('A3', '0.25'), ('Rest', '0.25'), ('A3', '0.75'), ('Rest', '0.75'), ('A3', '0.5'), ('A3', '1.0'), ('A3', '0.5'), ('A3', '0.5'), ('Rest', '0.5'), ('A3', '0.5'), ('A3', '1.0'), ('G#3', '0.5'), ('A3', '1.0'), ('A3', '0.75'), ('Rest', '1.25'), ('A3', '1.0'), ('G#3', '0.75'), ('Rest', '0.75'), ('A3', '0.75'), ('Rest', '0.75'), ('G#3', '1.0'), ('A3', '1.0'), ('G#3', '1.0'), ('G#3', '6.0'), ('G#3', '6.0'), ('G#3', '0.75'), ('Rest', '0.75'), ('F#3', '4.5'), ('F#3', '2.5'), ('Rest', '3.5')]


## 3. GPT를 사용해 기타 지판 추출

In [ ]:
from openai import OpenAI
import pandas as pd

df = pd.read_csv('handied_data/BP_CR2_01805.csv')
note = df['Pitch/Rest'].tolist()
beat = df['Duration'].tolist()

client = OpenAI(api_key=KEY)

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": f"계이름: {note}, 박자: {beat}"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://drive.google.com/uc?id=1SqqTmI3eJM2nzNuQbF6wOpyTs_qTDz73",
          },
        },
      ],
    },
    {
        "role": "user",
        "content":"기타 지판 위치를 json 형식으로 알려줘"
    }
  ]
)
print(response.choices[0].message.content)

In [6]:
tab_notes = [
    {"string": 6, "fret": 3, "note": "G2", "duration": 1},  # 6번 줄, 3번 프렛
    {"string": 5, "fret": 5, "note": "D3", "duration": 2},  # 5번 줄, 5번 프렛
    {"string": 4, "fret": 7, "note": "A3", "duration": 1},  # 4번 줄, 7번 프렛
]

In [8]:
from music21 import stream, note

# 타브 스트림 생성
tab_stream = stream.Stream()

# 타브 표기 데이터
tab_notes = [
    {"string": 6, "fret": 3, "note": "G2", "duration": 1},
    {"string": 5, "fret": 5, "note": "D3", "duration": 2},
    {"string": 4, "fret": 7, "note": "A3", "duration": 1},
]

# 타브 표기 추가
for tab_note in tab_notes:
    # note.Note 객체 생성
    n = note.Note(tab_note['note'])
    n.quarterLength = tab_note['duration']  # 음표 길이 설정

    # 타브 정보를 주석 또는 가사 형태로 추가
    n.lyric = f"String: {tab_note['string']}, Fret: {tab_note['fret']}"  # Lyric으로 추가

    tab_stream.append(n)

# 텍스트로 확인
tab_stream.show('text')


{0.0} <music21.note.Note G>
{1.0} <music21.note.Note D>
{3.0} <music21.note.Note A>


## 4. Lilypond 연동하여 파일 및 이미지 생성

In [19]:
def create_tab_ly_file(notes, filename):
    with open(filename, 'w') as file:
        file.write('\\version "2.24.2"\n')
        file.write('\\header {')
        file.write('  title = "Tablature Example"\n')
        file.write('}\n')
        file.write('\\score {\n')
        file.write('  \\new StaffGroup <<\n')
        file.write('    \\new Staff = "main" <<\n')
        file.write('      \\new Voice = "melody" \\relative c\' {\n')
        
        for pitch, duration in notes:
            if pitch.lower() == 'rest':
                file.write(f'r{duration} ')
            else:
                file.write(f'{pitch.lower()}{duration} ')

        file.write('\n      }\n')
        file.write('    >>\n')
        file.write('    \\new TabStaff = "tab" <<\n')
        file.write('      \\relative c\' {\n')
        
        for pitch, duration in notes:
            if pitch.lower() == 'rest':
                file.write(f'r{duration} ')
            else:
                file.write(f'{pitch.lower()}{duration} ')

        file.write('\n      }\n')
        file.write('    >>\n')
        file.write('  >>\n')
        file.write('}\n')

ly_file = 'output_tab.ly'
create_tab_ly_file(notes, ly_file)
print(f'LilyPond 파일이 {ly_file}에 저장되었습니다.')


LilyPond 파일이 output_tab.ly에 저장되었습니다.


In [14]:
import subprocess

def ly_to_png(ly_file_path, output_dir):

    try:
        subprocess.run(
            ["lilypond", "--png", "-o", output_dir, ly_file_path],
            check=True
        )
        print(f"PNG 파일이 {output_dir}에 생성되었습니다.")
    except FileNotFoundError:
        print("LilyPond가 설치되지 않았거나 PATH에 등록되지 않았습니다.")
    except subprocess.CalledProcessError as e:
        print("PNG 변환 중 오류가 발생했습니다:", e)

ly_file = "out.ly"
output_directory = "."

ly_to_png(ly_file, output_directory)


PNG 파일이 .에 생성되었습니다.


In [28]:
# import csv
# import subprocess
# from fractions import Fraction

# def read_csv(file_path):
#     """
#     CSV 파일 읽기
#     """
#     score_data = []
#     with open(file_path, 'r') as f:
#         reader = csv.DictReader(f)
#         for row in reader:
#             score_data.append({
#                 "pitch": row["Pitch/Rest"],
#                 "duration": row["Duration"]
#             })
#     return score_data


# def convert_duration(duration):
#     """
#     음표 길이를 LilyPond 형식으로 변환
#     Args:
#         duration (str): 원본 음표 길이
#     Returns:
#         str: LilyPond 형식의 음표 길이
#     """
#     try:
#         # 소수점 또는 분수 처리
#         if "/" in duration:
#             fraction = Fraction(duration)  # 분수 처리
#             if fraction.denominator > 32:  # 너무 세밀한 분수는 지원되지 않음
#                 return "4"  # 기본값
#             return f"{fraction.denominator // fraction.numerator}"
#         elif "." in duration:
#             value = float(duration)
#             if value == 0.5:
#                 return "8"  # 8분음표
#             elif value == 1.5:
#                 return "2."  # 점 2분음표
#             else:
#                 return str(int(4 / value))  # 4분음표 기준 비율
#         else:
#             return duration
#     except ValueError:
#         return "4"  # 기본값


# def convert_to_lilypond(score_data):
#     """
#     CSV 데이터를 LilyPond 형식으로 변환
#     """
#     lilypond_notes = []
#     for item in score_data:
#         pitch = item["pitch"].lower()
#         duration = convert_duration(item["duration"])

#         # 쉼표 처리
#         if pitch == "rest":
#             lilypond_note = f"r{duration}"
#         else:
#             lilypond_note = f"{pitch}{duration}"

#         lilypond_notes.append(lilypond_note)

#     # LilyPond 기본 템플릿
#     lilypond_template = r"""
#     \version "2.24.2"
#     \score {
#       <<
#         \new TabStaff {
#           \relative c {
#             """ + " ".join(lilypond_notes) + r"""
#           }
#         }
#       >>
#     }
#     """
#     return lilypond_template


# def save_lilypond_file(content, file_path):
#     """
#     .ly 파일 저장
#     """
#     with open(file_path, "w", encoding="utf-8") as f:
#         f.write(content)


# def generate_png_from_ly(ly_file_path, output_dir):
#     """
#     LilyPond로 PNG 변환
#     """
#     try:
#         subprocess.run(
#             ["lilypond", "--png", "-o", output_dir, ly_file_path],
#             check=True
#         )
#         print(f"PNG 파일이 {output_dir}에 생성되었습니다.")
#     except FileNotFoundError:
#         print("LilyPond가 설치되지 않았거나 PATH에 등록되지 않았습니다.")
#     except subprocess.CalledProcessError as e:
#         print("PNG 변환 중 오류가 발생했습니다:", e)


# # 실행 흐름
# csv_file = "handied_data/BP_CR2_01853.csv"  # CSV 파일 경로
# ly_file = "gptoutput.ly"  # 생성할 .ly 파일 경로
# output_directory = "."  # PNG 출력 디렉토리

# # 1. CSV 읽기
# score = read_csv(csv_file)

# # 2. LilyPond 코드 생성
# lilypond_code = convert_to_lilypond(score)

# # 3. .ly 파일 저장
# save_lilypond_file(lilypond_code, ly_file)

# # 4. PNG 변환
# generate_png_from_ly(ly_file, output_directory)


PNG 변환 중 오류가 발생했습니다: Command '['lilypond', '--png', '-o', '.', 'gptoutput.ly']' returned non-zero exit status 1.


In [22]:
# import subprocess

# def convert_ly_to_png(ly_file, output_dir):
#     command = f'lilypond -o {output_dir} {ly_file}'
#     result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
#     if result.returncode != 0:
#         print(f'오류 발생: {result.stderr}')
#     else:
#         print(f'PNG 파일이 {output_dir}.png에 저장되었습니다.')

# # 생성된 Ly 파일을 PNG로 변환
# png_output_dir = 'output_tab'
# convert_ly_to_png(ly_file, png_output_dir)


오류 발생: Processing `output_tab.ly'
Parsing...
output_tab.ly:9:2: error: not a duration
a
 30d5 a30d5 a30d25 r0d25 a30d5 a30d5 a30d5 a30d5 a30d5 a30d25 r1d25 a30d5 a31/3 a31/3 r1/12 a31/3 a30d25 r0d25 a30d75 r0d75 a30d5 a31d0 a30d5 a30d5 r0d5 a30d5 a31d0 g#30d5 a31d0 a30d75 r1d25 a31d0 g#30d75 r0d75 a30d75 r0d75 g#31d0 a31d0 g#31d0 g#36d0 g#36d0 g#30d75 r0d75 f#34d5 f#32d5 r3d5 

output_tab.ly:9:5: error: not a duration
a30d
    5 a30d5 a30d25 r0d25 a30d5 a30d5 a30d5 a30d5 a30d5 a30d25 r1d25 a30d5 a31/3 a31/3 r1/12 a31/3 a30d25 r0d25 a30d75 r0d75 a30d5 a31d0 a30d5 a30d5 r0d5 a30d5 a31d0 g#30d5 a31d0 a30d75 r1d25 a31d0 g#30d75 r0d75 a30d75 r0d75 g#31d0 a31d0 g#31d0 g#36d0 g#36d0 g#30d75 r0d75 f#34d5 f#32d5 r3d5 

output_tab.ly:9:8: error: not a duration
a30d5 a
       30d5 a30d25 r0d25 a30d5 a30d5 a30d5 a30d5 a30d5 a30d25 r1d25 a30d5 a31/3 a31/3 r1/12 a31/3 a30d25 r0d25 a30d75 r0d75 a30d5 a31d0 a30d5 a30d5 r0d5 a30d5 a31d0 g#30d5 a31d0 a30d75 r1d25 a31d0 g#30d75 r0d75 a30d75 r0d75 g#31d0 